# Electricity demand in Quebec

We now focus on electricity consumption in Quebec.



## Hydro-Québec open data

Since 2019, Hydro-Québec shares data on various aspects regarding its operations, at the page https://www.hydroquebec.com/documents-data/open-data/.

Here, we are interested in the historical demand: https://www.hydroquebec.com/documents-data/open-data/history-electricity-demand-quebec/. In particular, we can track the hourly total demand in Quebec, noting that
> Electricity demand for one hour corresponds to the total average demand during that hour.

> The data is determined at the end of a time period. For example, the average hourly demand associated with 2019‑01‑01 2:00 is the average of the data collected from 2019‑01‑01 1:05 to 2019‑01‑01 2:00.

We will analyze the data using Pandas and Plotly.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

We consider data from 2020 to 2023 (included).

In [ ]:
df = []

for i in range(2020,2024):
    s = "data\\"+str(i)+"-demande-electricite-quebec.xlsx"
    df.append(pd.read_excel(s))
    
demand_data = pd.concat(df)

In [ ]:
demand_data.info()

We split the date in two columns, one for the day, one for the hour.

In [ ]:
demand_data = demand_data.rename(columns={'Date': 'Datetime'})
demand_data['Day'] = pd.to_datetime(demand_data['Datetime']).dt.date
demand_data['Hour'] = pd.to_datetime(demand_data['Datetime']).dt.time
demand_data.info()

We now duplicate the dateframe in order to have a first look in the daily and hourly demands.

In [ ]:
daily = demand_data.copy()
hourly = demand_data.copy()

We first build the daily consumption data.

In [ ]:
daily = daily.groupby(by=["Day"])["Moyenne (MW)"].sum().reset_index()

daily.tail()

In [ ]:
plot = go.Figure(px.scatter(hourly, x='Hour', y='Moyenne (MW)'))

plot.update_layout(
    title = 'Hourly electricity demand',
    yaxis_title='Electricity consumption (MW)',
    xaxis_title='Hour',
)

plot.show()

Let's take a look at the average demand.

In [ ]:
hourly = hourly.groupby(by=["Hour"])["Moyenne (MW)"].mean().reset_index()

plot = go.Figure(px.line(hourly, x='Hour', y='Moyenne (MW)'))

plot.update_layout(
    title = 'Mean hourly electricity consumption',
    yaxis_title='Electricity consumption (MW)',
    xaxis_title='Hour',
)

plot.show()

We observe two electricity demand peaks, one in the morning, and one in the evening.

Let's have a look at the daily patterns.

In [ ]:
plot = go.Figure(px.line(daily, x='Day', y='Moyenne (MW)'))

# Update layout
plot.update_layout(
    title = 'Daily electricity consumption',
    yaxis_title='Electricity consumption (MW)',
    xaxis_title='Date',
)

# Show the plot
plot.show()

We can observe a seasonality effect, with a much higher consumption during the Winter. This suggests that the consumption is highly correlated with the temperature.

But we can face an atypical day, more difficult to forecast.

In [ ]:
demand_data['Day'] = demand_data['Day'].astype(str)
df = demand_data.loc[demand_data['Day'] == "2023-02-03"] 

plot = go.Figure(px.line(df, x='Hour', y='Moyenne (MW)'))

# Update layout
plot.update_layout(
    title = "A cold day (2023-02-03; BQ record)",
    yaxis_title='Electricity consumption (MW)',
    xaxis_title='Hour',
)

# Show the plot
plot.show()

## Weather information

Weather historical date can be retrieved from https://climate.weather.gc.ca/historical_data/search_historic_data_e.html

More explanations about the columns are available at https://www.canada.ca/en/environment-climate-change/services/climate-change/canadian-centre-climate-services/display-download/technical-documentation-hourly-data.html

Data are downloaded using the bash script

```{verbatin}
#!/bin/bash

station_ID=51157
start_year=2020
end_year=2024
path="weather_data"

for year in $(seq $start_year $end_year); do
    for month in $(seq 1 12); do
        wget -P ${path} --content-disposition "https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=${station_ID}&Year=${year}&Month=${month}&Day=14&timeframe=1&submit=Download+Data"
    done
done
```
Main components:
- "weather_data" is the path where the data files will be stored;
- stationID designs the weather data station.

We use the MONTREAL INTL A (51157) station and merge the monthly data in a single CSV file.

Now, we can read it.

In [ ]:
YUL_weather = pd.read_csv("data\\montreal_intl_a_weather.csv", dtype={"Precip. Amount Flag": "string", "Visibility Flag": "string"})

In [ ]:
# map column name
column_mapping = {
    "Temp (°C)": "temp",
    "Dew Point Temp (°C)": "rel_hum",
    "Wind Chill": "wind_chill",
    "Wind Spd (km/h)": "wind_speed",
    "Wind Dir (10s deg)": "wind_dir",
    "Weather": "condition",
    "Date/Time (LST)": "Datetime"
}

YUL_weather = YUL_weather.rename(columns=column_mapping)

YUL_weather.info()

We do some cleaning by first removing all columns with 0 entry.

In [ ]:
YUL_weather.dropna(how='all', axis=1, inplace=True)

We also drop columns with a unique value.

In [ ]:
for col in YUL_weather.columns:
    if len(YUL_weather[col].unique()) == 1:
        YUL_weather.drop(col,inplace=True,axis=1)

We now reformat the date and time column to make it similar to the demand dataframe.

In [ ]:
# YUL_weather['datetime'] = pd.to_datetime(YUL_weather['datetime'], format='mixed')
YUL_weather['Datetime'] = pd.to_datetime(YUL_weather['Datetime'])

Let's check the obtained dataframe.

In [ ]:
YUL_weather.tail()

In [ ]:
YUL_weather.info()

We align the date range with the electricity demand dataframe.

In [ ]:
start = "2020-01-01 01:00:00"
end = "2024-01-01 00:00:00"
YUL_weather.drop(YUL_weather[YUL_weather['Datetime'] < start].index, inplace=True)
YUL_weather.drop(YUL_weather[YUL_weather['Datetime'] > end].index, inplace=True)

In [ ]:
YUL_weather.info()

In [ ]:
YUL_weather.tail()

We merge the demand and weather database in order to analyze the relationships between the electricity consumption and the weather conditions.

In [ ]:
result = pd.merge(demand_data, YUL_weather, on="Datetime")

We are now in position to observe the relations between the weather and the demand. We will here focus on the temperature.

In [ ]:
plot = go.Figure(px.scatter(result, x='temp', y='Moyenne (MW)'))

# Update layout
plot.update_layout(
    title = 'Temperature and consumption',
    yaxis_title='Electricity consumption (MW)',
    xaxis_title='Temperature',
)

# Show the plot
plot.show()

Quick observations:
- cold temperatures lead to higher consumption, mainly due to the heating;
- we start to observe an effect due to air conditioning at high temperatures.